This notebook aims to assess the generalization accuracy of a generated suffix, assuming a data-split was used during training.

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from transformers import AutoTokenizer
import pandas as pd
import seaborn as sns
from types import SimpleNamespace
from datasets import Dataset
from os.path import join as oj
import pickle as pkl
import os
import dvu
dvu.set_style()
import analyze_utils
import sys
sys.path.append('..')
import data
from model_utils import prompt_classification

cols_output = ['final_answer_found',
               'final_model_queries', 'final_num_suffixes_checked']

# Load the results + get generated suffixes

In [2]:
# dir_single_query = '/home/chansingh/mntv1/single_query_math_9_17' # single-digit
# dir_single_query = '/home/chansingh/mntv1/single_query_math_double_digit'
# dir_single_query = '/home/chansingh/mntv1/single_query_math_long_suffs' # suffix results
dir_single_query = '/home/chansingh/mntv1/suffix_math_long_suffs_full' # suffix results
r = analyze_utils.load_results_and_cache(dir_single_query, save_file='r_long.pkl', only_keep_scalar=False)
r_single_query = pd.read_pickle(os.path.join(dir_single_query, 'r_long.pkl'))
r_single_query = analyze_utils.postprocess_results(r_single_query)

# dir_suffix_search = '/home/chansingh/mntv1/suffix_math_9_17' # single-digit
# dir_suffix_search = '/home/chansingh/mntv1/suffix_math_double_digit'
# dir_suffix_search = '/home/chansingh/mntv1/suffix_math_long_suffs'
dir_suffix_search = '/home/chansingh/mntv1/suffix_anli_long_suffs_full'
r_suffix_search = analyze_utils.load_results_and_cache(dir_suffix_search, save_file='r_long.pkl', only_keep_scalar=False)
r_suffix_search = pd.read_pickle(os.path.join(dir_suffix_search, 'r_long.pkl'))
r_suffix_search = analyze_utils.postprocess_results(r_suffix_search)

# r = r_single_query
r = pd.concat((r_single_query, r_suffix_search)).sort_values('checkpoint')
# r.to_pickle(f'../results/prompt_gen/results_suffix_math.pkl')

  6%|▌         | 3/54 [00:00<00:13,  3.74it/s]

skipping 032601ab0390a97e17260e692084b80ff478d0d6281f8aea285edc9a0e85374dvrwi
skipping 0524fc9903db96eaa2985f8a1225e9185286cd6eb370f2e56d4dffd08cd1ff45mkdj


100%|██████████| 40/40 [00:03<00:00, 12.15it/s]


# Look at top suffix candidates

In [ ]:
r[r.max_num_tokens == 6]

In [5]:
r[['suffix_str_added', 'final_answer_added']]

,suffix_str_added,final_answer_added
32,"[, the, a, 1, an, 2, the sum, the resul...",sum
25,"[, the, a, it, "", an, the input, the st...",the opposite
30,"[, the, a, 1, 0, to, the result, the in...",divide
20,"[, the, a, an, "", it, the first, the st...",NaN
28,"[, the, -, a, 0, 1, the result, the sum...",the difference
...,...,...
51,"[, the, to, a, 1, 0, the input, the num...",the sum
5,"[, the, to, a, it, an, the value, the i...",a value of the opposite
10,"[, the, to, it, a, an, the value, the i...",NaN
31,"[, the, to, a, 1, 0, the input, the num...",the sum


In [6]:
args = r.iloc[1]



In [ ]:
r0 = r[(r.max_num_tokens == 6) * (r.n_shots == 1) * (r.checkpoint == 'EleutherAI/gpt-j-6B')]
for i in range(r0.shape[0]):
    row = r0.iloc[i]
    print('task', row.task_name)
    candidates, probs = analyze_utils.get_top_candidates_and_probs_suff(row)
    for j in range(5):
        print('\t', candidates[j], probs[j])

# Assess accuracy of the found suffixes

In [38]:
args = r.iloc[3]

In [39]:
np.random.seed(args.seed)
(dset, dset_test), check_answer_func, descr = data.get_data(
            args, args.task_name, n_shots=args.n_shots, train_split_frac=args.train_split_frac)

In [43]:
checkpoint = 'EleutherAI/gpt-j-6B' # 'gpt2-medium'
model = prompt_classification.create_model(checkpoint)

In [ ]:
loss, acc = prompt_classification.test_model_on_task_with_prefix(
    dset=dset, model=model, prefix=f'Return the antonym of the input: ', multi_token=True,
)